In [ ]:
import netCDF4
import numpy as np
import pandas as pd
import geopandas as gpd

from datetime import datetime, timedelta
from tqdm import tqdm
from shapely.geometry import Point

from scipy.stats import skew, kurtosis, entropy

from scipy.fft import fft
from sklearn.preprocessing import MinMaxScaler
import os

from pycaret.classification import setup, compare_models, tune_model, finalize_model, save_model, plot_model, evaluate_model, dashboard, save_experiment, blend_models

import pyarrow as pa
import pyarrow.parquet as pq
from collections import Counter
from sklearn.model_selection import train_test_split
import multiprocessing


In [ ]:

class SurfaceTypeUtils:
    surface_type_dict = {
        -1: "Ocean",
        0: "NaN",
        1: "Artifical",
        2: "Barely vegetated",
        3: "Inland water",
        4: "Crop",
        5: "Grass",
        6: "Shrub",
        7: "Forest"
    }
    ddm_antennas = {
        0: 'None',
        1: 'Zenith',
        2: 'LHCP',
        3: 'RHCP',
    }


In [ ]:
ROOT_DIR= 'D:/data/RONGOWAI_L1_SDR_V1.0/'

In [ ]:
class GeoUtils:
    def __init__(self, world_shapefile_path):
        self.world = gpd.read_file(world_shapefile_path)

    @staticmethod
    def add_seconds(time, seconds):
        timestamp = datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
        new_timestamp = timestamp + timedelta(seconds=seconds)
        return new_timestamp.strftime("%Y-%m-%d %H:%M:%S")

    def is_land(self, lat, lon):
        point = Point(lon, lat)
        return any(self.world.contains(point))

    @staticmethod
    def check_ocean_and_land(lst):
        has_ocean = -1 in lst
        has_land = any(1 <= num <= 7 for num in lst)
        return has_ocean and has_land

    @staticmethod
    def fill_and_filter(arr):
        mask_all_nan = np.all(np.isnan(arr), axis=(2, 3))
        arr_filled = arr.copy()
        for i in range(arr.shape[0]):
            nan_indices = np.where(mask_all_nan[i])[0]
            if len(nan_indices) > 0:
                valid_indices = np.where(~mask_all_nan[i])[0]
                if len(valid_indices) > 0:
                    mean_matrix = np.nanmean(arr[i, valid_indices, :, :], axis=0)
                    arr_filled[i, nan_indices, :, :] = mean_matrix
        mask_discard = np.all(mask_all_nan, axis=1)
        arr_filtered = arr_filled[~mask_discard]
        return arr_filtered, list(np.where(mask_discard.astype(int) == 1)[0])

In [ ]:

class GeoHelper:
    def __init__(self, world):
        self.world = world

    @staticmethod
    def add_seconds(time, seconds):
        timestamp = datetime.strptime(time, "%Y-%m-%d %H:%M:%S")
        new_timestamp = timestamp + timedelta(seconds=seconds)
        return new_timestamp.strftime("%Y-%m-%d %H:%M:%S")

    @staticmethod
    def fill_and_filter(arr):
        mask_all_nan = np.all(np.isnan(arr), axis=(2, 3))
        arr_filled = arr.copy()
        for i in range(arr.shape[0]):
            nan_indices = np.where(mask_all_nan[i])[0]
            if len(nan_indices) > 0:
                valid_indices = np.where(~mask_all_nan[i])[0]
                if len(valid_indices) > 0:
                    mean_matrix = np.nanmean(arr[i, valid_indices, :, :], axis=0)
                    arr_filled[i, nan_indices, :, :] = mean_matrix
        mask_discard = np.all(mask_all_nan, axis=1)
        arr_filtered = arr_filled[~mask_discard]
        return arr_filtered, list(np.where(mask_discard.astype(int) == 1)[0])


In [ ]:
class NetCDFPreprocessor:

    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.netcdf_file_list = os.listdir(root_dir)

    def preprocess(self, f):
        
        raw_counts = f.variables['raw_counts']
        raw_counts = np.array(raw_counts)

        # Calcolo distanza tra il punto speculare e l'aereo
        ac_alt_2d = np.repeat(np.array(f.variables['ac_alt'])[:, np.newaxis], 20, axis=1)
        distance_2d = (ac_alt_2d - f.variables['sp_alt'][:]) / np.cos(np.deg2rad(f.variables['sp_inc_angle'][:]))

        # Seleziona gli indici dove sp_rx_gain_copol > 5, sp_rx_gain_xpol > 5 e ddm_snr > 0 e distanza tra punto speculare e antenna > 2000 e < 10000
        copol = f.variables['sp_rx_gain_copol'][:]
        xpol = f.variables['sp_rx_gain_xpol'][:]
        snr = f.variables['ddm_snr'][:]
        dist = distance_2d[:]

        keep_mask = (copol >= 5) & (xpol >= 5) & (snr > 0) & ((dist >= 2000) & (dist <= 10000)) & (~np.isnan(copol.data) & ~np.isnan(xpol.data) & ~np.isnan(snr.data) & ~np.isnan(dist.data))
        to_keep_indices = np.argwhere(keep_mask)
        
        filtered_raw_counts = [raw_counts[i, j] for i, j in to_keep_indices]
        #filtered_raw_counts_arr = np.array(filtered_raw_counts)

        output_array = np.full(raw_counts.shape, np.nan, dtype=np.float32)

        # Inserisci i dati filtrati nelle posizioni di to_keep_indices
        for idx, (i, j) in enumerate(to_keep_indices):
            output_array[i, j] = filtered_raw_counts[idx]

        raw_counts_filtered = output_array.copy()
        del output_array

        ddm_data_dict = {
            'Raw_Counts': raw_counts_filtered.reshape(raw_counts_filtered.shape[0]*raw_counts_filtered.shape[1], raw_counts_filtered.shape[2], raw_counts_filtered.shape[3]),
        }
        keep_indices = np.where(
            np.all(~np.isnan(ddm_data_dict['Raw_Counts']), axis=(1, 2)) & (np.sum(ddm_data_dict['Raw_Counts'], axis=(1, 2)) > 0)
        )[0]
        fit_data = np.array([ddm_data_dict['Raw_Counts'][f].ravel() for f in keep_indices])
        surface_types = f.variables["sp_surface_type"][:]
        surface_types = np.nan_to_num(surface_types, nan=0)
        surface_types_unravelled = surface_types.ravel()

        label_data = surface_types_unravelled
        label_data = [label_data[lab] for lab in range(len(label_data)) if lab in keep_indices]

        assert np.array(fit_data).shape[0] == len(label_data), \
            f"Shape mismatch: fit_data {np.array(fit_data).shape[0]}, label_data {len(label_data)}"

        return fit_data, label_data

    def process_all_files(self, chunk_size = int, sample_fraction = float, remove_chunks= bool):
        
        full_data = []
        full_labels = []
        #counter = 0
        for file_name in tqdm(self.netcdf_file_list, desc="Processing files"):
            if not file_name.endswith('.nc'):
                continue
            try:
                f = netCDF4.Dataset(f'{self.root_dir}{file_name}')
                data, labels = self.preprocess(f)
                full_data.append(data)
                full_labels.append(labels)
            except Exception as e:
                print(f"Error processing file {file_name}: {e}")
                continue
            #counter += 1
            #if counter == 100:  # Limita a 50 file per il caricamento
            #    break
        
        # Trova gli indici degli elementi di full_data con seconda dimensione uguale a 200
        valid_indices = [i for i, arr in enumerate(full_data) if arr.ndim == 2 if arr.shape[1] == 200]

        # Applica la selezione a full_data e full_labels
        full_data_clean = [full_data[i] for i in valid_indices]
        full_labels_clean = [full_labels[i] for i in valid_indices]

        del full_data, full_labels # Libera memoria

        # Chunking 
        
        os.makedirs('processed_data/multiclass', exist_ok=True)

        chunk_size = chunk_size # dimensione del chunk in numero di campioni
        sample_fraction = sample_fraction  # frazione di dati da campionare per ogni chunk

        full_data_sampled = []
        full_labels_sampled = []

        num_chunks = int(np.ceil(len(full_data_clean) / chunk_size))
        for idx in range(num_chunks):
            start = idx * chunk_size
            end = min((idx + 1) * chunk_size, len(full_data_clean))
            chunk_data = np.vstack(full_data_clean[start:end])
            chunk_labels = np.hstack(full_labels_clean[start:end])
            print('-' * 50)
            
            print(f"Chunk {idx + 1}/{num_chunks} processed with shape {chunk_data.shape} and labels shape {chunk_labels.shape}")
            
            # Salva ogni chunk come file parquet separato
            fit_data_df = pd.DataFrame(chunk_data)
            labels_df = pd.DataFrame(chunk_labels, columns=['label'])

            table_fit = pa.Table.from_pandas(fit_data_df, preserve_index=False)
            table_labels = pa.Table.from_pandas(labels_df, preserve_index=False)

            pq.write_table(
                table_fit,
                f'processed_data/multiclass/fit_data_chunk_{idx}.parquet',
                compression='zstd',
                use_dictionary=True,
            )
            pq.write_table(
                table_labels,
                f'processed_data/multiclass/labels_chunk_{idx}.parquet',
                compression='zstd',
                use_dictionary=True,
            )
            del table_fit, table_labels, fit_data_df, labels_df

            # Trova le classi più rare (meno rappresentate)
            label_counts = Counter(chunk_labels)
            min_count = min(label_counts.values())
            rare_classes = [cls for cls, count in label_counts.items() if count == min_count]

            # Seleziona tutte le occorrenze delle classi rare
            rare_indices = np.isin(chunk_labels, rare_classes)
            X_rare = chunk_data[rare_indices]
            y_rare = chunk_labels[rare_indices]

            # Per le altre classi, esegui un campionamento casuale per raggiungere la frazione desiderata
            other_indices = ~rare_indices
            X_other = chunk_data[other_indices]
            y_other = chunk_labels[other_indices]

            _, X_other_sampled, _, y_other_sampled = train_test_split(
                X_other, y_other,
                test_size=sample_fraction,
                stratify=y_other,
                random_state=42
            )

            # Combina i dati delle classi rare con quelli campionati delle altre classi
            X_sampled = np.vstack([X_rare, X_other_sampled])
            y_sampled = np.hstack([y_rare, y_other_sampled])

            print(f"Distribuzione etichette prima del campionamento nel chunk {idx + 1}: {Counter(chunk_labels)}")
            print(f"Distribuzione etichette dopo campionamento nel chunk {idx + 1}: {Counter(y_sampled)}")

            print('-' * 50)
            
            
            
            full_data_sampled.append(X_sampled)
            full_labels_sampled.append(y_sampled)

        full_data_sampled_stratified = np.vstack(full_data_sampled)
        full_labels_sampled_stratified = np.hstack(full_labels_sampled)

        print(f"Distribuzione totale etichette dopo stratificazione: {Counter(full_labels_sampled_stratified)}")

        
        print(f"Shape of sampled data after chunking and sampling: {np.array(full_data_sampled_stratified).shape}")
        print(f"Shape of sampled labels after chunking and sampling: {np.array(full_labels_sampled_stratified).shape}")
        
        # Crea la cartella processed_data se non esiste
        os.makedirs('processed_data/multiclass', exist_ok=True)

        # Salva fit_data in formato parquet ottimizzato
        fit_data_df = pd.DataFrame(full_data_sampled_stratified)
        table_fit = pa.Table.from_pandas(fit_data_df, preserve_index=False)
        pq.write_table(
            table_fit,
            'processed_data/multiclass/fit_data_multiclass_raw_counts_only.parquet',
            compression='zstd',
            use_dictionary=True,
            
        )

        # Salva labels in formato parquet ottimizzato
        labels_df = pd.DataFrame(full_labels_sampled_stratified, columns=['label'])
        table_labels = pa.Table.from_pandas(labels_df, preserve_index=False)
        pq.write_table(
            table_labels,
            'processed_data/multiclass/labels_multiclass_raw_counts_only.parquet',
            compression='zstd',
            use_dictionary=True,
            
        )

        del fit_data_df, labels_df, table_fit, table_labels
        # Remove all chunk parquet files if flag is set
        if remove_chunks:
            try:
                chunk_dir = 'processed_data/multiclass'
                for fname in os.listdir(chunk_dir):
                    if fname.startswith('fit_data_chunk_') or fname.startswith('labels_chunk_'):
                        os.remove(os.path.join(chunk_dir, fname))
                print("All chunk files removed.")
            except Exception as e:
                print(f"Error removing chunk files: {e}")

        return full_data_sampled_stratified, full_labels_sampled_stratified

In [ ]:

class DDMFeatureExtractor:
    def __init__(self):
        pass
    def gini(self, array):
            """Calcola il coefficiente di Gini (disuguaglianza)"""
            array = np.sort(array)
            index = np.arange(1, array.shape[0] + 1)
            return (np.sum((2 * index - array.shape[0] - 1) * array)) / (array.shape[0] * np.sum(array))    
    def extract_ddm_features(self, fit_data: np.ndarray) -> pd.DataFrame:

        """
        Estrae features dettagliate da raw_counts DDM (shape: n_samples x 200)
        """
        features = []

        for row in tqdm(fit_data, desc="Extracting DDM features"):
            f = {}
            x = np.array(row, dtype=np.float64) + 1e-10  # evita log(0)

            # 1. Statistiche base
            f['mean'] = np.mean(x)
            f['std'] = np.std(x)
            f['min'] = np.min(x)
            f['max'] = np.max(x)
            f['median'] = np.median(x)
            f['range'] = np.max(x) - np.min(x)
            f['skew'] = skew(x)
            f['kurtosis'] = kurtosis(x)
            f['entropy'] = entropy(x)
            f['gini'] = self.gini(x)

            # 2. Posizionali
            f['peak_index'] = np.argmax(x)
            f['peak_value'] = np.max(x)
            f['center_of_mass'] = np.sum(np.arange(len(x)) * x) / np.sum(x)
            f['inertia'] = np.sum(((np.arange(len(x)) - f['center_of_mass'])**2) * x)

            # 3. Segmentazione
            thirds = np.array_split(x, 3)
            for i, part in enumerate(thirds):
                f[f'sum_third_{i+1}'] = np.sum(part)
                f[f'mean_third_{i+1}'] = np.mean(part)
                f[f'max_third_{i+1}'] = np.max(part)
            
            windows = np.array_split(x, 5)
            for i, w in enumerate(windows):
                f[f'mean_w{i+1}'] = np.mean(w)
                f[f'std_w{i+1}'] = np.std(w)
                f[f'max_w{i+1}'] = np.max(w)

            # 4. Derivate e cambiamenti
            dx = np.diff(x)
            f['mean_diff'] = np.mean(dx)
            f['std_diff'] = np.std(dx)
            f['max_diff'] = np.max(dx)
            f['min_diff'] = np.min(dx)
            f['n_positive_diff'] = np.sum(dx > 0)
            f['n_negative_diff'] = np.sum(dx < 0)
            f['n_zero_diff'] = np.sum(dx == 0)

            # 5. Autocorrelazioni (lag 1-3)
            for lag in range(1, 4):
                ac = np.corrcoef(x[:-lag], x[lag:])[0, 1] if len(x) > lag else np.nan
                f[f'autocorr_lag{lag}'] = ac

            # 6. FFT (spettro frequenze)
            spectrum = np.abs(fft(x))
            half_spectrum = spectrum[:len(spectrum)//2]  # simmetrico
            f['fft_peak_freq'] = np.argmax(half_spectrum)
            f['fft_max'] = np.max(half_spectrum)
            f['fft_median'] = np.median(half_spectrum)
            f['fft_mean'] = np.mean(half_spectrum)

            
            #Aggiungi le statistiche dei quadranti e del centro
            ddm = row.reshape(10, 20)  # 10x20

            # Quadranti
            q1 = ddm[:5, :10].ravel()
            q2 = ddm[:5, 10:].ravel()
            q3 = ddm[5:, :10].ravel()
            q4 = ddm[5:, 10:].ravel()
            # Quadrante centrale (4x8 centrale)
            center = ddm[3:7, 6:14].ravel()
            
            # Statistiche dei quadranti 
            f['q1_mean'] = np.mean(q1)
            f['q2_mean'] = np.mean(q2)      
            f['q3_mean'] = np.mean(q3)
            f['q4_mean'] = np.mean(q4)
            f['center_mean'] = np.mean(center)
            f['q1_std'] = np.std(q1)
            f['q2_std'] = np.std(q2)
            f['q3_std'] = np.std(q3)
            f['q4_std'] = np.std(q4)
            f['center_std'] = np.std(center)
            f['q1_min'] = np.min(q1)
            f['q2_min'] = np.min(q2)
            f['q3_min'] = np.min(q3)
            f['q4_min'] = np.min(q4)
            f['center_min'] = np.min(center)
            f['q1_max'] = np.max(q1)
            f['q2_max'] = np.max(q2)
            f['q3_max'] = np.max(q3)
            f['q4_max'] = np.max(q4)
            f['center_max'] = np.max(center)
            f['q1_median'] = np.median(q1)
            f['q2_median'] = np.median(q2)
            f['q3_median'] = np.median(q3)
            f['q4_median'] = np.median(q4)
            f['center_median'] = np.median(center)
            f['q1_range'] = np.max(q1) - np.min(q1)
            f['q2_range'] = np.max(q2) - np.min(q2)
            f['q3_range'] = np.max(q3) - np.min(q3)
            f['q4_range'] = np.max(q4) - np.min(q4)
            f['center_range'] = np.max(center) - np.min(center)
            f['q1_skew'] = skew(q1)
            f['q2_skew'] = skew(q2)
            f['q3_skew'] = skew(q3)
            f['q4_skew'] = skew(q4)
            f['center_skew'] = skew(center)
            f['q1_kurtosis'] = kurtosis(q1)
            f['q2_kurtosis'] = kurtosis(q2)
            f['q3_kurtosis'] = kurtosis(q3)
            f['q4_kurtosis'] = kurtosis(q4)
            f['center_kurtosis'] = kurtosis(center)
            f['q1_entropy'] = entropy(q1 + 1e-10)
            f['q2_entropy'] = entropy(q2 + 1e-10)
            f['q3_entropy'] = entropy(q3 + 1e-10)
            f['q4_entropy'] = entropy(q4 + 1e-10)
            f['center_entropy'] = entropy(center + 1e-10)
            f['q1_gini'] = self.gini(q1)
            f['q2_gini'] = self.gini(q2)
            f['q3_gini'] = self.gini(q3)
            f['q4_gini'] = self.gini(q4)
            f['center_gini'] = self.gini(center)

            # Statistiche di confronto tra quadranti e centro
            
            # Differenze tra media dei quadranti e centro
            f['q1_center_mean_diff'] = f['q1_mean'] - f['center_mean']
            f['q2_center_mean_diff'] = f['q2_mean'] - f['center_mean']
            f['q3_center_mean_diff'] = f['q3_mean'] - f['center_mean']
            f['q4_center_mean_diff'] = f['q4_mean'] - f['center_mean']

            # Differenze tra std dei quadranti e centro
            f['q1_center_std_diff'] = f['q1_std'] - f['center_std']
            f['q2_center_std_diff'] = f['q2_std'] - f['center_std']
            f['q3_center_std_diff'] = f['q3_std'] - f['center_std']
            f['q4_center_std_diff'] = f['q4_std'] - f['center_std']

            # Differenze tra max dei quadranti e centro
            f['q1_center_max_diff'] = f['q1_max'] - f['center_max']
            f['q2_center_max_diff'] = f['q2_max'] - f['center_max']
            f['q3_center_max_diff'] = f['q3_max'] - f['center_max']
            f['q4_center_max_diff'] = f['q4_max'] - f['center_max']

            # Differenze tra min dei quadranti e centro
            f['q1_center_min_diff'] = f['q1_min'] - f['center_min']
            f['q2_center_min_diff'] = f['q2_min'] - f['center_min']
            f['q3_center_min_diff'] = f['q3_min'] - f['center_min']
            f['q4_center_min_diff'] = f['q4_min'] - f['center_min']

            # Differenze tra entropia dei quadranti e centro
            f['q1_center_entropy_diff'] = f['q1_entropy'] - f['center_entropy']
            f['q2_center_entropy_diff'] = f['q2_entropy'] - f['center_entropy']
            f['q3_center_entropy_diff'] = f['q3_entropy'] - f['center_entropy']
            f['q4_center_entropy_diff'] = f['q4_entropy'] - f['center_entropy']

            # Differenze tra gini dei quadranti e centro
            f['q1_center_gini_diff'] = f['q1_gini'] - f['center_gini']
            f['q2_center_gini_diff'] = f['q2_gini'] - f['center_gini']
            f['q3_center_gini_diff'] = f['q3_gini'] - f['center_gini']
            f['q4_center_gini_diff'] = f['q4_gini'] - f['center_gini']

            # Differenze tra skewness dei quadranti e centro
            f['q1_center_skew_diff'] = f['q1_skew'] - f['center_skew']
            f['q2_center_skew_diff'] = f['q2_skew'] - f['center_skew']
            f['q3_center_skew_diff'] = f['q3_skew'] - f['center_skew']
            f['q4_center_skew_diff'] = f['q4_skew'] - f['center_skew']

            # Differenze tra kurtosis dei quadranti e centro
            f['q1_center_kurtosis_diff'] = f['q1_kurtosis'] - f['center_kurtosis']
            f['q2_center_kurtosis_diff'] = f['q2_kurtosis'] - f['center_kurtosis']
            f['q3_center_kurtosis_diff'] = f['q3_kurtosis'] - f['center_kurtosis']
            f['q4_center_kurtosis_diff'] = f['q4_kurtosis'] - f['center_kurtosis']

            features.append(f)
            

        return features
    
    

In [ ]:

class ModelTrainer:
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.final_model = None

    def visualize_model_performances(self, model):
        #try:
        #    print("Creazione del dashboard del modello...")
        #    dashboard(model)
        #except Exception as e:
        #    print(f"Errore durante la creazione del dashboard: {e}")

        try:
            print("Valutazione del modello...")
            evaluate_model(model)
        except Exception as e:
            print(f"Errore durante la valutazione del modello: {e}")
        
        try:
            print("Creazione della matrice di confusione del modello...")
            plot_model(model, plot='confusion_matrix', save=False)
        except Exception as e:
            print(f"Errore durante la creazione della matrice di confusione: {e}")

        try:
            print("Creazione del grafico delle feature del modello...")
            plot_model(model, plot='feature_all', save=False)
        except Exception as e:
            print(f"Errore durante la creazione del grafico delle feature: {e}")
        
        try:
            print("Creazione del report di classificazione del modello...")
            plot_model(model, plot='class_report', save=False)
        except Exception as e:
            print(f"Errore durante la creazione del report di classificazione: {e}")
        
        try:
            print("Creazione del grafico PR del modello...")
            plot_model(model, plot='pr', save=False)
        except Exception as e:
            print(f"Errore durante la creazione del grafico PR: {e}")
        
        try:
            print("Calibrazione del modello...")
            plot_model(model, plot='calibration', save=False)
        except Exception as e:
            print(f"Errore durante la creazione del grafico di calibrazione: {e}")


    def search_and_train_model(self, model_search=True):
        os.environ["PYCARET_CUSTOM_LOGGING_LEVEL"] = "CRITICAL"
        if model_search:
            scaler = MinMaxScaler()
            fit_data_scaled = scaler.fit_transform(self.data)
            clf_exp = setup(data=fit_data_scaled,
                        target=self.labels,
                        #pca=True,
                        #pca_method='incremental',
                        use_gpu=True,
                        feature_selection=True,
                        #n_features_to_select=.4,
                        )
            
            best_models = compare_models(n_select=3, 
                                         exclude=['gbc', 'dummy', 'qda', 'lda', 'nb', 'svm'],
                                         sort='Accuracy',
                                         )

            best_model = best_models[0]

            print(f"Il modello migliore è: {best_model}")
            
            print("Ottimizzazione degli iperparametri del modello migliore...")
            tuned_model = tune_model(best_model,
                                    optimize='Accuracy',
                                    n_iter=10,
                                    search_library='optuna',
                                    search_algorithm='tpe',
                                    choose_better=True)
            print("Valutazione del modello ottimizzato:")

            best_params = tuned_model.get_params()

            print("Migliori iperparametri trovati:")
            for param, value in best_params.items():
                print(f"{param}: {value}")

            self.final_model = finalize_model(tuned_model)

            save_model(self.final_model, 'best_binary_classification_model', prep_pipeline=True)
            print("Modello finale salvato come 'best_binary_classification_model'.")
            # Salva l'esperimento
            save_experiment('binary_classification_experiment')
            print("Esperimento salvato come 'binary_classification_experiment'.")
            self.visualize_model_performances(self.final_model)
      
        
    def train_ensemble_model(self, n_sample_per_class=int):  
        os.environ["PYCARET_CUSTOM_LOGGING_LEVEL"] = "CRITICAL"
        # Campiona  righe con distribuzione bilanciata tra le classi di labels_df  
        sampled_indices = (
            self.labels.groupby(self.labels.iloc[:, 0])
            .apply(lambda x: x.sample(n=n_sample_per_class, random_state=42))
            .index.get_level_values(1)
        )
        features_df = self.data.loc[sampled_indices].reset_index(drop=True)
        labels_df = self.labels.loc[sampled_indices].reset_index(drop=True)

        scaler = MinMaxScaler()
        fit_data_scaled = scaler.fit_transform(features_df)
        clf_exp = setup(data=fit_data_scaled,
                    target=labels_df['0'],
                    #pca=True,
                    #pca_method='incremental',
                    use_gpu=True,
                    feature_selection=True,
                    n_features_to_select=.4,
                    )
        
        best_models = compare_models(n_select=3, 
                                        exclude=['gbc', 'dummy', 'qda', 'lda', 'nb', 'svm'],
                                        sort='Accuracy',
                                        )  
        
        print("Ensembling dei migliori modelli...")
        best_models = [model for model in best_models if model is not None]
        print(f"Modelli selezionati per l'ensembling: {best_models}")
        ensembled_models = blend_models(best_models, 
                                        method='soft', 
                                        fold=5, 
                                        optimize='Accuracy', 
                                        )
        if ensembled_models is None:
            print("Nessun modello ensembled creato. Verifica i modelli selezionati.")
            return
        
        print(f"Modelli ensembled creati: {ensembled_models}")

        self.visualize_model_performances(ensembled_models)

        self.final_ensembled_model = finalize_model(ensembled_models)
        save_model(self.final_ensembled_model, 'best_binary_classification_ensembled_model', prep_pipeline=True)
        save_experiment('binary_classification_ensembled_experiment')
            

In [ ]:
os.environ['HDF5_USE_FILE_LOCKING'] = 'FALSE'

In [ ]:
ROOT_DIR = 'D:/data/RONGOWAI_L1_SDR_V1.0/'

read_from_backup = False  
if read_from_backup:
    
    fit_data_pl = pd.read_parquet('processed_data/multiclass/fit_data_multiclass_raw_counts_only.parquet')
    labels_pl = pd.read_parquet('processed_data/multiclass/labels_multiclass_raw_counts_only.parquet')

    fit_data = fit_data_pl.to_numpy()
    labels = labels_pl['label'].to_numpy()
else:
    preprocessor = NetCDFPreprocessor(root_dir=ROOT_DIR)
    fit_data, labels = preprocessor.process_all_files(chunk_size=250, sample_fraction=0.1, remove_chunks=True)

try:
    del fit_data_pl, labels_pl
except NameError:
    print("fit_data_pl and labels_pl not defined, skipping deletion.")

In [ ]:
fit_data = fit_data[:3000000]
labels = labels[:3000000]

In [ ]:
preprocessor = NetCDFPreprocessor(root_dir=ROOT_DIR)
features_extractor = DDMFeatureExtractor()

In [ ]:
from joblib import Parallel, delayed

def extract_ddm_features_row(row):
    return features_extractor.extract_ddm_features(np.array([row]))

combined_features = Parallel(n_jobs=12, backend="loky")(delayed(extract_ddm_features_row)(row) for row in tqdm(fit_data, desc="Estrazione features"))

In [ ]:
flat_features = [row[0] if isinstance(row, list) and len(row) > 0 else row for row in combined_features]
del combined_features
FEATURES = list(flat_features[0].keys())
combined_features = np.array([[row[key] for key in FEATURES] for row in flat_features])
del flat_features
combined_features.shape

# Controlla infiniti o valori troppo grandi per float64
mask_finite = np.isfinite(combined_features).all(axis=1) & (np.abs(combined_features) < np.finfo(np.float64).max).all(axis=1)

fit_data_with_features_clean = combined_features[mask_finite]
labels_clean = labels[mask_finite]

In [ ]:
os.makedirs('processed_data/multiclass/data_w_features', exist_ok=True)
pd.DataFrame(fit_data_with_features_clean, columns=FEATURES).to_parquet('processed_data/multiclass/data_w_features/fit_data_multiclass_stat_features_only.parquet', index=False)
pd.DataFrame(labels_clean).to_parquet('processed_data/multiclass/data_w_features/labels_multiclass_stat_features_only.parquet', index=False)

In [ ]:
# Usa la lista FEATURES già creata sopra per i nomi delle colonne
fit_data_with_features_df = pd.DataFrame(fit_data_with_features_clean, columns=FEATURES)

del fit_data_with_features_clean
fit_data_with_features_df.head()

In [ ]:
model_trainer = ModelTrainer(data=fit_data_with_features_df, labels=labels_clean)
model_trainer.train(model_search=True)

TARGET MAPPING = -1.0: 0, 1.0: 1, 2.0: 2, 3.0: 3, 4.0: 4, 5.0: 5, 6.0: 6, 7.0: 7

2